In [3]:
import pandas as pd
import tarfile
import io

df_mc_meta=pd.read_csv('/data2st1/junyi/methlyatlas/mCseq/CEMBA.mC.Metadata/CEMBA.mC.Metadata.csv')

In [ ]:
df_mc_meta

In [ ]:
df_mc_meta.head(10).to_csv('/data2st1/junyi/methlyatlas/mCseq/CEMBA.mC.Metadata/CEMBA.mC.Metadata.head10.csv',index=False)

In [ ]:
import h5py

# Open the HDF5 file in read mode
with h5py.File("/data2st1/junyi/methlyatlas/atac/catlas.org/catlas_downloads/mousebrain/snap/CEMBA180205_4A.snap", 'r') as file:
    # List all datasets in the HDF5 file
    print("Datasets in the HDF5 file:")
    for dataset in file.keys():
        print(dataset)

    # Read a specific dataset
    dataset_name = 'your_dataset_name'  # Replace 'your_dataset_name' with the actual dataset name
    data = file[dataset_name][:]  # Read the dataset into a NumPy array

    # Perform operations on the data as needed
    print("Shape of the dataset:", data.shape)
    print("Data from the dataset:")
    print(data)


In [ ]:
df_meta_integrateion = pd.read_csv("/data2st1/junyi/methlyatlas/mCseq/mC-ATAC.Integration/mC-ATAC.Integration.csv")

In [ ]:
df_meta_integrateion.head(10)

In [ ]:
import h5py
filename = "/data2st1/junyi/methlyatlas/atac/catlas.org/catlas_downloads/mousebrain/snap/CEMBA180123_2A.snap"

with h5py.File(filename, 'r') as file:
    # List all groups in the file
    print("Groups in the HDF5 file:")
    for group in file.keys():
        print(group)


In [ ]:
import snapatac2 as snap

snap.__version__


In [ ]:
adata=snap.read("/data2st1/junyi/methlyatlas/atac/catlas.org/catlas_downloads/mousebrain/snap/CEMBA180123_2A.snap",backend="hdf5")

In [15]:
import zarr 
import numpy as np
z = zarr.open('/data2st1/junyi/methlyatlas/mCseq/data.nemoarchive.org/biccn/grant/u19_cemba/ecker/epigenome/cellgroup/mCseq3/mouse/processed/counts/CEMBA.snmC.L4RegionAgg.zarr', mode='r')


"_ARRAY_DIMENSIONS": [
    "L4Region",
    "chrom100k",
    "mc_type",
    "count_type"
],


In [ ]:
z['/count_type/'][:]

In [ ]:
z['/mc_type/'][:]

In [ ]:
z['/geneslop2k-vm23_da/'][:100,:100,1,0].max()

In [5]:
from ALLCools.mcds import MCDS

import pandas as pd


In [2]:
var_dim = "chrom100k"
mcds = MCDS.open('/data2st1/junyi/methlyatlas/mCseq/data.nemoarchive.org/biccn/grant/u19_cemba/ecker/epigenome/cellgroup/mCseq3/mouse/processed/counts/CEMBA.snmC.L4RegionAgg.zarr',
                   obs_dim='L4Region',var_dim='chrom100k')


In [ ]:
mcds

In [ ]:
var_dim = "chrom100k"
mcds.add_feature_cov_mean(var_dim=var_dim)


In [77]:
from wmb import cemba, mm10

In [78]:
black_list_path = mm10.ENCODE_BLACKLIST_PATH


In [ ]:
black_list_path

In [ ]:
mcds = mcds.remove_black_list_region(
    black_list_path=black_list_path,
    f=0.2  # Features having overlap > f with any black list region will be removed.
)


In [ ]:
mcds = mcds.remove_chromosome(['chrM', 'chrX', 'chrY'])

In [82]:
with open('output/FeatureList.BasicFilter.txt', 'w') as f:
    for var in mcds.get_index(mcds.var_dim).astype(str):
        f.write(var + '\n')

In [83]:
mcds.add_mc_frac(
normalize_per_cell=True,  # after calculating mC frac, per cell normalize the matrix
    clip_norm_value=10  # clip outlier values above 10 to 10
)


In [ ]:
mch_pattern = 'CHN'
mcg_pattern = 'CGN'
n_top_feature = 20000

mch_hvf = mcds.calculate_hvf_svr(var_dim=var_dim,
                                 mc_type=mch_pattern,
                                 n_top_feature=n_top_feature,
                                 plot=True)


In [ ]:
mcg_hvf = mcds.calculate_hvf_svr(var_dim=var_dim,
                                 mc_type=mcg_pattern,
                                 n_top_feature=n_top_feature,
                                 plot=True)


In [86]:
gene_meta_path = '/home/junyichen/code/whole_mouse_brain/wmb/files/gencode.vM22.annotation.gene.flat.tsv.gz'

In [ ]:
chrom_to_remove = ['chrM', 'chrX', 'chrY']


In [6]:
var_dim = "geneslop2k-vm23"
mcds = MCDS.open('/data2st1/junyi/methlyatlas/mCseq/data.nemoarchive.org/biccn/grant/u19_cemba/ecker/epigenome/cellgroup/mCseq3/mouse/processed/counts/CEMBA.snmC.L4RegionAgg.zarr',
                   obs_dim='L4Region',var_dim=var_dim)


In [ ]:
abs(mcds['geneslop2k-vm23_start']-mcds['geneslop2k-vm23_end']).min()

In [8]:
gene_meta_path = '/home/junyichen/code/whole_mouse_brain/wmb/files/modified_gencode.vM23.primary_assembly.annotation.gene.flat.tsv.gz'


In [9]:
gene_meta = pd.read_csv(gene_meta_path, index_col='gene_id', sep='\t')


In [ ]:
gene_meta

In [ ]:
import pybedtools

genes_to_skip = set()
chrom_to_remove = ['chrM']

# skip smaller genes mostly covered by a larger gene, e.g., a miRNA within a protein coding gene.
# F=0.9 means > 90% of gene_b is overlapped with gene_a, in this case, we only keep gene_a for DMG test
gene_bed = pybedtools.BedTool.from_dataframe(
    gene_meta.reset_index()[['chrom', 'start', 'end', 'gene_id']])
mapped_bam = gene_bed.map(b=gene_bed, c=4, o='distinct', F=0.9)
for _, (*_, gene_a, gene_b_str) in mapped_bam.to_dataframe().iterrows():
    for gene_b in gene_b_str.split(','):
        if gene_b != gene_a:
            genes_to_skip.add(gene_b)

# remove certain chromosomes
genes_to_skip |= set(gene_meta.index[gene_meta['chrom'].isin(chrom_to_remove)])
use_features = gene_meta.index[~gene_meta.index.isin(genes_to_skip)]
print(f'{use_features.size} features remained')

In [ ]:
use_features

In [ ]:

min_cov = 5

mcds.add_feature_cov_mean()

feature_cov_mean = mcds.coords[f'{var_dim}_cov_mean'].to_pandas()
use_features &= feature_cov_mean[feature_cov_mean > min_cov].index


In [ ]:
use_features

In [15]:
use_gene_meta = gene_meta.loc[use_features]


In [ ]:
use_gene_meta

In [18]:
use_gene_meta.to_csv('output/GeneMetadata.csv.gz')


In [17]:
gtf_path = '/data2st1/junyi/ref/gencode.vM23.chr_patch_hapl_scaff.annotation.gtf.gz'

In [45]:
gene_meta = pd.read_csv(gtf_path, sep='\t',skiprows=5,header=None)


In [ ]:
gene_meta

In [ ]:
mcds["L4Region"]

In [55]:
df_region = pd.read_csv('/data2st1/junyi/methlyatlas/atac/Subclass.names.map.csv.gz',index_col=0)

In [ ]:
df_region

In [57]:
df_meta_integrateion = pd.read_csv("/data2st1/junyi/methlyatlas/mCseq/CEMBA.mC.Metadata/CEMBA.mC.Metadata.csv")

In [59]:
df_meta_integrateion.drop_duplicates(subset='CellGroup',inplace=True)

In [64]:
df_meta_integrateion.set_index('CellGroup',inplace=True)

In [ ]:
df_meta_integrateion.loc[mcds["L4Region"]]

In [ ]:
df_meta_integrateion

In [ ]:
import pyBigWig


In [171]:
bw = pyBigWig.open("/data2st1/junyi/methlyatlas/atac/catlas.org/renlab_downloads/wholemousebrain/bigwig_macs2/001_CLA-EPd-CTX_Car3_Glut.bw")

In [ ]:
bw.isBigWig()

In [ ]:
use_gene_meta

In [192]:
stats =[]
for gene in use_gene_meta.index:
    chrom = use_gene_meta.loc[gene, 'chrom']
    start = use_gene_meta.loc[gene, 'start']
    end = use_gene_meta.loc[gene, 'end']
    
    values = bw.values(chrom, start-2000, start)
    arr_values = np.array(values)
    arr_values = np.nan_to_num(arr_values)
    stats.append(arr_values.sum())

In [ ]:
values

In [ ]:
import matplotlib.pyplot as plt

plt.hist(stats, bins=100, color='skyblue', edgecolor='black')

# Add labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.ylim(0,1000)
plt.title('Histogram of Random Data')


In [ ]:
np.array(stats).max()

In [177]:
import numpy as np
array=np.array(bw.values("chr1", 3205901-2000, 3205901))

In [ ]:
array = np.nan_to_num(array, nan=0).mean()
array

In [ ]:
arr = array
result = arr[(arr > 0) & (arr < 1)]
result

In [197]:
bw_atac = pyBigWig.open("/data2st1/junyi/methlyatlas/atac/catlas.org/renlab_downloads/wholemousebrain/bigwig_macs2/001_CLA-EPd-CTX_Car3_Glut.bw")

In [ ]:
values = bw_atac.values(chrom, 3205901, 3671498)
np.array(values).max()

In [181]:
stats =[]
for gene in use_gene_meta.index:
    chrom = use_gene_meta.loc[gene, 'chrom']
    start = use_gene_meta.loc[gene, 'start']
    end = use_gene_meta.loc[gene, 'end']
    
    values = bw.values(chrom, start-2000, start)
    arr_values = np.array(values)
    arr_values = np.nan_to_num(arr_values)
    stats.append(arr_values.mean())

In [1]:
import matplotlib.pyplot as plt
import anndata as adata
import scanpy as sc
from wmb import cemba, mm10
from ALLCools.mcds import MCDS
from ALLCools.clustering import significant_pc_test
from ALLCools.plot import *
import pandas as pd
from harmonypy import run_harmony

In [2]:
adata = sc.read_h5ad('output/merged-clustering.h5ad')

In [ ]:
sc.pl.umap(adata, color='MajorRegion', size=0.5, palette='tab20')

In [ ]:
interactive_scatter(data=adata,
                    hue='MajorRegion',
                    coord_base='umap',
                    max_points=10000)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4), dpi=300)
_ = categorical_scatter(data=adata,
                        ax=ax,
                        coord_base='umap',
                        hue='MajorRegion',
                        text_anno='MajorRegion',
                        show_legend=True)


In [10]:
df_mc_meta=pd.read_csv('/data2st1/junyi/methlyatlas/mCseq/CEMBA.mC.Metadata/CEMBA.mC.Metadata.csv')

In [ ]:
df_mc_meta.columns

In [ ]:
df_mc_meta.MajorRegion.unique()

In [ ]:
df_mc_meta.SubRegion.unique()

In [17]:
PFC_AMY_subregions = ['MOs','PL.ORB','PL,ACA,ILA','CTXsp,sAMY']
selected_cells = df_mc_meta[(df_mc_meta.SubRegion.isin(PFC_AMY_subregions)) | (df_mc_meta.MajorRegion.isin(["HPF"]))].index
selected_meta = df_mc_meta.loc[selected_cells]

In [ ]:
selected_meta

In [ ]:
df_umap = pd.read_csv('/data2st1/junyi/methlyatlas/mCseq/CEMBA.mC.Coordinates/CEMBA.mC.Coordinates.csv.gz',index_col=0)

In [ ]:
df_umap

In [43]:
var_dim = 'geneslop2k-vm23'
mcds = MCDS.open('/data2st1/junyi/methlyatlas/mCseq/data.nemoarchive.org/biccn/grant/u19_cemba/ecker/epigenome/cellgroup/mCseq3/mouse/processed/counts/CEMBA.snmC.mcds',
                   obs_dim='cell',var_dim=var_dim,use_obs=selected_meta.index)

In [28]:
sampled_df = selected_meta.groupby('SubRegion', group_keys=False).count()

In [35]:
sampled_df = selected_meta.sample(n=25000, random_state=42)  # Setting a random state for reproducibility


In [49]:
selected_df_meta = sampled_df.merge(df_umap,left_on="cell",right_index=True)